In [11]:
from pathlib import Path
from skimage.io import imread, imsave
from cellpose import models, io, utils
import os
import numpy as np

In [27]:
parent_path = r'D:\images\tnia-python-images\imagesc\2024_10_03_cellpose_ladybugs'
overlay_path = r'D:\images\tnia-python-images\imagesc\2024_10_03_cellpose_ladybugs\overlay_pretraineed'
model_path = os.path.join(parent_path, 'models')

model = models.CellposeModel(gpu=True, model_type=None, pretrained_model= os.path.join(model_path, 'models', 'cellposetry_again'))
#model = models.Cellpose(gpu=True, model_type=None, pretrained_model= os.path.join(model_path, 'models', 'cellpose_20241005_085232'))
stop=5


In [26]:
if not Path(overlay_path).exists():
    Path(overlay_path).mkdir()

files = Path(parent_path).glob('*.jpeg')

for file in files:
    print(file)
    img = imread(file)
    masks = model.eval(img, diameter=30, channels=[1, 0], cellprob_threshold=0.0, flow_threshold = 1.0)

    img_ = img#//img.max()

    outlines = utils.masks_to_outlines(masks[0])
    overlay = mask_overlay(img_, masks[0])
    imsave(overlay_path + '\\' + file.name, overlay)
    


D:\images\tnia-python-images\imagesc\2024_10_03_cellpose_ladybugs\33584593_52760085.jpeg
D:\images\tnia-python-images\imagesc\2024_10_03_cellpose_ladybugs\38920793_61915904.jpeg
D:\images\tnia-python-images\imagesc\2024_10_03_cellpose_ladybugs\4841329_5877801.jpeg


In [17]:
masks[0].shape

(2056, 2464)

In [9]:
import numpy

def mask_overlay(img, masks, colors=None):
    """Overlay masks on image (set image to grayscale).

    Args:
        img (int or float, 2D or 3D array): Image of size [Ly x Lx (x nchan)].
        masks (int, 2D array): Masks where 0=NO masks; 1,2,...=mask labels.
        colors (int, 2D array, optional): Size [nmasks x 3], each entry is a color in 0-255 range.

    Returns:
        RGB (uint8, 3D array): Array of masks overlaid on grayscale image.
    """
    if colors is not None:
        if colors.max() > 1:
            colors = np.float32(colors)
            colors /= 255
        colors = utils.rgb_to_hsv(colors)
    if img.ndim > 2:
        img = img.astype(np.float32).mean(axis=-1)
    else:
        img = img.astype(np.float32)

    HSV = np.zeros((img.shape[0], img.shape[1], 3), np.float32)
    HSV[:, :, 2] = np.clip((img / 255. if img.max() > 1 else img) * 1.5, 0, 1)
    hues = np.linspace(0, 1, masks.max() + 1)[np.random.permutation(masks.max())]
    for n in range(int(masks.max())):
        ipix = (masks == n + 1).nonzero()
        if colors is None:
            HSV[ipix[0], ipix[1], 0] = hues[n]
        else:
            HSV[ipix[0], ipix[1], 0] = colors[n, 0]
        HSV[ipix[0], ipix[1], 1] = 1.0
    RGB = (utils.hsv_to_rgb(HSV) * 255).astype(np.uint8)
    return RGB